In [1]:
# Install required packages
!pip install transformers datasets torch torchmetrics

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torchmetrics

In [3]:
import json

# Load the dataset
def load_squad_format(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
       data = json.load(f)

    contexts = []
    questions = []
    answers = []

    for item in data["data"]:
        for paragraph in item["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                for answer in qa["answers"]:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer["text"])

    return contexts, questions, answers

# Load train and validation datasets
train_contexts, train_questions, train_answers = load_squad_format('/kaggle/input/dataset-2/train.json')
val_contexts, val_questions, val_answers = load_squad_format('/kaggle/input/dataset-2/val.json')
test_contexts, test_questions, test_answers = load_squad_format('/kaggle/input/dataset-2/test.json')

In [4]:
import pandas as pd
df_train=pd.DataFrame()
df_train["context"]=train_contexts
df_train["questions"]=train_questions
df_train["answers"]=train_answers


df_test=pd.DataFrame()
df_test["context"]=test_contexts
df_test["questions"]=test_questions
df_test["answers"]=test_answers

df_val=pd.DataFrame()
df_val["context"]=val_contexts
df_val["questions"]=val_questions
df_val["answers"]=val_answers

df_val.head(5)

,context,questions,answers
0,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,ما هي غابات ميومبو وأين توجد؟\n,هي أراضي عُشبية استوائية وشبه استوائية بيولوجي...
1,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,ما هي المناطق البيئية الأربعة التي تميز غابات ...,يُمكن تصنيف غابات ميومبو على أنها جافةً أو رط...
2,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,كيف يتم تصنيف غابات ميومبو إلى أغاب ورطبة؟\n,هي تلك التي تستقبل أكثر من 1000 مم سنويًا لهطو...
3,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,كيف تساعد غابات ميومبو في توفير سُبل العيش للس...,الذين يعتمدون على الموارد المتاحة من الغابات
4,غابات ميومبو هي أراضي عُشبية استوائية وشبه است...,ما هي المنتجات غير الخشبية التي يمكن العثور عل...,مثل الفاكهة والعسل وعلف الماشية وحطب الوقود


In [5]:
print(f"Contexts: {len(train_contexts)}")
print(f"Questions: {len(train_questions)}")
print(f"Answers: {len(train_answers)}")

Contexts: 64782
Questions: 64782
Answers: 64782


In [6]:
from transformers import AutoTokenizer


# Load the pre-trained Arabic GPT-2 model and tokenizer
model_name = "aubmindlab/aragpt2-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

In [7]:

# Add a new [PAD] token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_data(contexts, questions, answers, tokenizer, max_length=128):
    input_texts = [
        f"السؤال: {str(q).encode('utf-8').decode('utf-8')} "
        f"السياق: {str(c).encode('utf-8').decode('utf-8')}"
        for q, c in zip(questions, contexts)
    ]
    
    # Ensure answers are properly encoded
    outputs = [str(a).encode('utf-8').decode('utf-8') for a in answers]

    inputs = tokenizer(
        input_texts,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    outputs = tokenizer(
        outputs,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Set labels to outputs for GPT-2 fine-tuning
    inputs["labels"] = outputs["input_ids"]
    return inputs


# Tokenize train and validation data
train_data = tokenize_data(train_contexts, train_questions, train_answers, tokenizer)
val_data = tokenize_data(val_contexts, val_questions, val_answers, tokenizer)
test_data = tokenize_data(test_contexts, test_questions, test_answers, tokenizer)


In [8]:
print(f"Input IDs shape: {train_data['input_ids'].shape}")
print(f"Labels shape: {train_data['labels'].shape}")

Input IDs shape: torch.Size([64782, 128])
Labels shape: torch.Size([64782, 128])


In [9]:
import torch
from torch.utils.data import Dataset
class QADataset(Dataset):
    def __init__(self, inputs):
        self.input_ids = inputs["input_ids"]
        self.attention_mask = inputs["attention_mask"]
        self.labels = inputs["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

# Create datasets
train_dataset = QADataset(train_data)
val_dataset = QADataset(val_data)
test_dataset = QADataset(test_data)

In [10]:
model.resize_token_embeddings(len(tokenizer))

Embedding(64001, 768)

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,  # Ensure the model aligns with the tokenizer
    padding="max_length",  # Force padding to the max sequence length
)

In [34]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./arabic-qa-gpt2-new",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
     # Add gradient accumulation to handle memory
    gradient_accumulation_steps=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    eval_steps=500,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    learning_rate=5e-5,
    fp16=True,  # Use mixed precision training
    dataloader_pin_memory=False,  # Prevent memory issues
    remove_unused_columns=False,  # Keep all columns
    report_to="none"
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import Trainer, TrainingArguments, TrainerCallback
import torch

class TrainingMonitorCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\n=== Starting Epoch: {state.epoch + 1}/{int(args.num_train_epochs)} ===\n")

    def on_step_begin(self, args, state, control, **kwargs):
        print(f"--- Step: {state.global_step}/{state.max_steps} ---")

    def on_train_batch_begin(self, args, state, control, train_dataloader, **kwargs):
        print(f"Processing Batch {state.global_step}...")

# Initialize trainer
def initialize_trainer(model, train_dataset, val_dataset):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[TrainingMonitorCallback()],
        data_collator=data_collator
    )
    return trainer

In [14]:
def train_model(train_dataset, val_dataset, test_dataset):


    # Initialize and start training
    trainer = initialize_trainer(model, train_dataset, val_dataset)
    trainer.train()

    # Evaluate on test set
    test_results = trainer.evaluate(test_dataset)
    print(f"Test Results: {test_results}")

    # Save the fine-tuned model
    model.save_pretrained("./arabic-qa-gpt2-finetuned")
    tokenizer.save_pretrained("./arabic-qa-gpt2-finetuned")

    return test_results



In [15]:
results = train_model(train_dataset, val_dataset, test_dataset)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.



=== Starting Epoch: 1/1 ===

--- Step: 0/2024 ---


/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss
500,1.016900,0.971867
1000,0.916400,0.933964
1500,0.910000,0.917943
2000,0.880600,0.915721


--- Step: 1/2024 ---
--- Step: 2/2024 ---
--- Step: 3/2024 ---
--- Step: 4/2024 ---
--- Step: 5/2024 ---
--- Step: 6/2024 ---
--- Step: 7/2024 ---
--- Step: 8/2024 ---
--- Step: 9/2024 ---
--- Step: 10/2024 ---
--- Step: 11/2024 ---
--- Step: 12/2024 ---
--- Step: 13/2024 ---
--- Step: 14/2024 ---
--- Step: 15/2024 ---
--- Step: 16/2024 ---
--- Step: 17/2024 ---
--- Step: 18/2024 ---
--- Step: 19/2024 ---
--- Step: 20/2024 ---
--- Step: 21/2024 ---
--- Step: 22/2024 ---
--- Step: 23/2024 ---
--- Step: 24/2024 ---
--- Step: 25/2024 ---
--- Step: 26/2024 ---
--- Step: 27/2024 ---
--- Step: 28/2024 ---
--- Step: 29/2024 ---
--- Step: 30/2024 ---
--- Step: 31/2024 ---
--- Step: 32/2024 ---
--- Step: 33/2024 ---
--- Step: 34/2024 ---
--- Step: 35/2024 ---
--- Step: 36/2024 ---
--- Step: 37/2024 ---
--- Step: 38/2024 ---
--- Step: 39/2024 ---
--- Step: 40/2024 ---
--- Step: 41/2024 ---
--- Step: 42/2024 ---
--- Step: 43/2024 ---
--- Step: 44/2024 ---
--- Step: 45/2024 ---
--- Step: 46/2024 -

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Test Results: {'eval_loss': 0.8607094883918762, 'eval_runtime': 99.2738, 'eval_samples_per_second': 140.732, 'eval_steps_per_second': 17.598, 'epoch': 0.9997530254383798}


In [16]:
wandb_key = '7fce0d03dfc1e78160dcfe859983acc039b0a5be'

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class MultiHeadDifferentialAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.d_model = config.n_embd
        self.n_head = config.n_head
        assert self.d_model % self.n_head == 0
        
        self.d_head = self.d_model // self.n_head
        
        # Parameters for Q1, Q2, K1, K2, V
        self.W_q = nn.Linear(self.d_model, 2 * self.d_model)
        self.W_k = nn.Linear(self.d_model, 2 * self.d_model)
        self.W_v = nn.Linear(self.d_model, 2 * self.d_model)
        
        # Learnable parameters for Î»
        self.lambda_q1 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_k1 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_q2 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_k2 = nn.Parameter(torch.randn(self.d_head))
        self.lambda_init = 0.8
        
        self.dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        self.proj = nn.Linear(self.d_model, self.d_model)
        
        # For compatibility with GPT-2
        self.register_buffer("bias", torch.tril(torch.ones(config.n_ctx, config.n_ctx)).view(
            1, 1, config.n_ctx, config.n_ctx))

    def compute_lambda(self, batch_size):
        lambda_term1 = torch.exp(self.lambda_q1 @ self.lambda_k1)
        lambda_term2 = torch.exp(self.lambda_q2 @ self.lambda_k2)
        lambda_val = lambda_term1 - lambda_term2 + self.lambda_init
        return lambda_val.view(1, 1, 1).expand(batch_size, -1, -1)

    def split_heads(self, x):
        new_shape = x.size()[:-1] + (self.n_head, self.d_head)
        x = x.view(*new_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states,
        layer_past=None,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        use_cache=False,
        output_attentions=False,
    ):
        batch_size, seq_length, _ = hidden_states.size()

        # Compute Q1, Q2, K1, K2, V
        qkv_outputs = []
        for proj in [self.W_q, self.W_k, self.W_v]:
            output = proj(hidden_states)
            if proj != self.W_v:
                output = output.chunk(2, dim=-1)
                qkv_outputs.extend(output)
            else:
                qkv_outputs.append(output)
        
        Q1, Q2, K1, K2, V = qkv_outputs
        
        # Split heads
        Q1, Q2 = map(self.split_heads, [Q1, Q2])
        K1, K2 = map(self.split_heads, [K1, K2])
        V = self.split_heads(V)
        
        # Handle past key values
        if layer_past is not None:
            past_k1, past_k2, past_v = layer_past
            K1 = torch.cat([past_k1, K1], dim=-2)
            K2 = torch.cat([past_k2, K2], dim=-2)
            V = torch.cat([past_v, V], dim=-2)
        
        if use_cache:
            present = (K1, K2, V)
        else:
            present = None

        # Scale attention scores
        scaling = math.sqrt(self.d_head)
        
        # Compute attention scores for both pairs
        attn_1 = torch.matmul(Q1, K1.transpose(-2, -1)) / scaling
        attn_2 = torch.matmul(Q2, K2.transpose(-2, -1)) / scaling
        
        # Apply causal mask
        causal_mask = self.bias[:, :, :seq_length, :K1.size(-2)]
        attn_1 = attn_1.masked_fill(causal_mask == 0, float('-inf'))
        attn_2 = attn_2.masked_fill(causal_mask == 0, float('-inf'))
        
        # Apply attention mask if provided
        if attention_mask is not None:
            attn_1 = attn_1 + attention_mask
            attn_2 = attn_2 + attention_mask
        
        # Compute softmax
        attn_1 = F.softmax(attn_1, dim=-1)
        attn_2 = F.softmax(attn_2, dim=-1)
        
        # Apply dropout
        attn_1 = self.dropout(attn_1)
        attn_2 = self.dropout(attn_2)
        
        # Compute lambda
        lambda_val = self.compute_lambda(batch_size)
        
        # Compute differential attention
        y = torch.matmul(attn_1, V) - lambda_val * torch.matmul(attn_2, V)
        
        # Restore sequence dimension and combine heads
        y = y.permute(0, 2, 1, 3).contiguous()
        y = y.view(batch_size, seq_length, self.d_model)
        
        # Final projection and dropout
        y = self.resid_dropout(self.proj(y))
        
        outputs = (y, present)
        if output_attentions:
            outputs += ((attn_1, attn_2),)
        
        return outputs  # output, present, (attentions)

In [55]:
from transformers import AutoModelForCausalLM
saved_model = AutoModelForCausalLM.from_pretrained("/kaggle/working/arabic-qa-gpt2-finetuned")

In [58]:
from transformers import AutoModelForCausalLM, pipeline

qa_pipeline = pipeline(
    task="text-generation",
    model=saved_model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

context = "اخترع ألكسندر غراهام بيل الهاتف في عام 1876."
question = "من هو مخترع الهاتف؟"
input_text = f"{question} {context}"

response = qa_pipeline(
    input_text,
    max_length=120,
    num_return_sequences=1,
    pad_token_id=tokenizer.pad_token_id
)
print(response[0]['generated_text'])

من هو مخترع الهاتف؟ اخترع ألكسندر غراهام بيل الهاتف في عام 1876.�ُ���  «��)ا� من على آخر (� حتى إلى" -ور.ع هذا كانتة!هر كان قد أي الاسم جائزةين 2 هي واحد : ر اسم " و وهو بين كود S وهيس كلمة ور لو فيه وما مهم أن أو ذلك بعض وجود ولماذا وكأنهاها وقد مثل غير البعض له يمكن التي يحدث عن استخدامح هذه لأن كما'وقفة الوصف “ ومن للع فين يطرق� دور وحتى وع ، وصل مشاهد الباب ع ان باسم اي المجال لقبut اعتبر الذيصف مع شهادة 3 تقييم محمد وضع لش التعليم رد عرضريم كور يصل 4 وصلت تكريم وعودة است لر ووصلت وصارت ابنها نال نقابة حقوق وحقوق. والحقوقصاف حماية حق وحق وحماية ومنح منح وجائزة وسام التكريم حصل مصير المصير طيار الطيار الطيران الجوي طائرات الجوية الطائرات للطائرات للطيران طائرة طائرار الطائرة طيران جوية الجو الجوي المطر احتج احتج احتج اض انف اد اد غكري « كادكوريونيدا... كوم ورد نير تهديد تهديد تهديد انتهاك تهديد تهديد تحذير تهديد تهديد عد « تهديد تهديد إنذار « تهديد اقتراح « تهديد الانذار « تحذير تحذير الحد الحد الحد تحذير التحذير تحذير تحذير منع تحذير تحذير تحذير حرم حرم مقام حرم حرم الحرم ذكر نبهr

In [51]:
def replace_attention_layers(model, ratio=0.3):
    """
    Replace a portion of the model's attention layers with differential attention

    Args:
        model: GPT-2 model
        ratio: Portion of layers to replace (between 0.25 and 0.5)
    """
    assert 0.25 <= ratio <= 0.5, "Ratio must be between 0.25 and 0.5"

    n_layers = len(model.transformer.h)

    n_replace = int(n_layers * ratio)

    layers_to_replace = torch.randperm(n_layers)[:n_replace].sort()[0].tolist()

    for layer_idx in layers_to_replace:
        layer = model.transformer.h[layer_idx]
        # Replace the attention module
        layer.attn = MultiHeadDifferentialAttention(model.config)

    return model, layers_to_replace

In [30]:
#new_model = replace_attention_layers(saved_model)

In [ ]:
# Usage example
if __name__ == "__main__":
    # Paths to your data files
    "
    
    # Train the modified model
    
    
    # Optional: Compare original and modified model outputs
    original_model, tokenizer = load_finetuned_model()
    modified_model = AutoModelForCausalLM.from_pretrained("./arabic-qa-gpt2-differential-final")
    
    # Example comparison
    question = "من هو مخترع الهاتف؟"
    context = "اخترع ألكسندر غراهام بيل الهاتف في عام 1876."
    
    comparison = compare_models(
        question, context, 
        original_model, modified_model, 
        tokenizer
    )
    print("\nComparison of model outputs:")
    print(comparison)

In [52]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from torch.utils.data import Dataset

import json

 

model1 = AutoModelForCausalLM.from_pretrained("./arabic-qa-gpt2-finetuned")

tokenizer1 = AutoTokenizer.from_pretrained("./arabic-qa-gpt2-finetuned")

 

# Tokenize train and validation data

train_data = tokenize_data(train_contexts, train_questions, train_answers, tokenizer1)

val_data = tokenize_data(val_contexts, val_questions, val_answers, tokenizer1)

test_data = tokenize_data(test_contexts, test_questions, test_answers, tokenizer1)




# Modify model with differential attention

def prepare_modified_model(original_model, replacement_ratio=0.3):

    # Import our previously defined differential attention class

    model, replaced_layers = replace_attention_layers(original_model, ratio=replacement_ratio)

    print(f"Replaced attention in layers: {replaced_layers}")

    return model

 

# Training arguments for the modified model

def get_training_args():

    return TrainingArguments(

        output_dir="./arabic-qa-gpt2-differential",

        num_train_epochs=1,  # Fewer epochs since model is already fine-tuned

        per_device_train_batch_size=8,

        per_device_eval_batch_size=8,

        warmup_steps=200,

        weight_decay=0.01,

        logging_dir="./logs_differential",

        logging_steps=50,

        save_steps=500,

        eval_steps=500,

        evaluation_strategy="steps",

        load_best_model_at_end=True,

        metric_for_best_model="loss",

        greater_is_better=False,

        # Add gradient clipping to help stabilize training

        max_grad_norm=1.0,

        # Add learning rate scheduling

        learning_rate=1e-5,
        
        report_to="none"
    )

 

# Main training function for modified model

def train_modified_model(train_data, val_data, test_data,model):

 

   

    # 2. Replace attention layers

    modified_model = prepare_modified_model(model)

   

    # 3. Load datasets (using our previous ArabicQADataset class)

    train_dataset = QADataset(train_data)

    val_dataset = QADataset(val_data)

    test_dataset = QADataset(test_data)

   

    # 4. Initialize trainer

    training_args = get_training_args()

    trainer = Trainer(

        model=modified_model,

        args=training_args,

        train_dataset=train_dataset,

        eval_dataset=val_dataset,

    )

   

    # 5. Train the model

    print("Starting training of modified model...")

    trainer.train()

   

    # 6. Evaluate on test set

    print("Evaluating on test set...")

    test_results = trainer.evaluate(test_dataset)

    print(f"Test Results: {test_results}")

   

    # 7. Save the modified model

    modified_model.save_pretrained("./arabic-qa-gpt2-differential-final")

    tokenizer1.save_pretrained("./arabic-qa-gpt2-differential-final")

   

    return test_results

 

# Function to compare model outputs

def compare_models(question, context, original_model, modified_model, tokenizer):

    input_text = f"{question} [SEP] {context} [SEP]"

    inputs = tokenizer(input_text, return_tensors="pt")

   

    # Generate from original model

    original_output = original_model.generate(

        **inputs,

        max_length=150,

        num_return_sequences=1,

        no_repeat_ngram_size=2

    )

   

    # Generate from modified model

    modified_output = modified_model.generate(

        **inputs,

        max_length=150,

        num_return_sequences=1,

        no_repeat_ngram_size=2

    )

   

    return {

        "Original": tokenizer.decode(original_output[0]),

        "Modified": tokenizer.decode(modified_output[0])

    }

 

In [53]:
results = train_modified_model(train_data, val_data, test_data,model1)

Replaced attention in layers: [0, 4, 8]


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Starting training of modified model...


RuntimeError: shape '[8, 128, 12, 64]' is invalid for input of size 1572864